In [8]:
pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("df_working.csv")

/var/folders/_t/mfgk4xxn1hb8w2n3w5x65snw0000gp/T/ipykernel_16723/908440793.py:1: DtypeWarning: Columns (5,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("df_working.csv")


In [5]:
# Models to try
models = {
    # 'Linear Regression': LinearRegression(),
    # 'Ridge Regression': Ridge(),
    # 'Lasso Regression': Lasso(),
    'Random Forest': RandomForestRegressor(),
    # 'Decision Tree': DecisionTreeRegressor(),
    # 'AdaBoost': AdaBoostRegressor(),
    'XGBoost': XGBRegressor(verbosity=0)
    # 'Support Vector': SVR()
}

In [8]:
feature_cols = [
    'VISITTYPE', 'VSTCTGRY', 'DOCATLOC', 'DRSPLTY_M18',
    'MEDPTYPE_M18', 'SEEDOC_M18', 'SURGPROC', 'MEDPRESC', 'TELEHEALTHFLAG',
    'LABTEST_M18', 'XRAYS_M18', 'MRI_M18', 'EKG_M18', 'OBDATEYR', 'OBDATEMM', 'insurance'
]

df[feature_cols] = df[feature_cols].fillna('Missing')
df[feature_cols] = df[feature_cols].astype(str)
df_model = df[feature_cols + ['total_charge', 'out_of_pocket', 'COST_COVERED']]

X = df_model[feature_cols]
print(X)

targets = {
    'Total Cost': df_model['total_charge'],
    'Out-of-Pocket': df_model['out_of_pocket'],
    'Cost Covered': df_model['COST_COVERED']
}

categorical_cols = X.select_dtypes(include='object').columns.tolist() + [
    'VISITTYPE', 'VSTCTGRY', 'DOCATLOC', 'DRSPLTY_M18',
    'MEDPTYPE_M18', 'SEEDOC_M18', 'SURGPROC', 'MEDPRESC', 'TELEHEALTHFLAG',
    'LABTEST_M18', 'XRAYS_M18', 'MRI_M18', 'EKG_M18', 'OBDATEYR', 'OBDATEMM'
]


cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_pipeline, categorical_cols)
    ],
    remainder='passthrough'
)

        VISITTYPE VSTCTGRY DOCATLOC DRSPLTY_M18 MEDPTYPE_M18 SEEDOC_M18  \
0         Missing      1.0  Missing         8.0      Missing        1.0   
1         Missing      2.0      2.0     Missing          8.0        2.0   
2         Missing      1.0  Missing        14.0      Missing        1.0   
3         Missing      2.0  Missing        14.0      Missing        1.0   
4         Missing      2.0  Missing        14.0      Missing        1.0   
...           ...      ...      ...         ...          ...        ...   
1007501   Missing      1.0  Missing         6.0      Missing        1.0   
1007502   Missing      1.0      2.0     Missing         11.0        2.0   
1007503   Missing      1.0  Missing         8.0      Missing        1.0   
1007504   Missing      1.0  Missing         8.0      Missing        1.0   
1007505   Missing      5.0  Missing         8.0      Missing        1.0   

        SURGPROC MEDPRESC TELEHEALTHFLAG LABTEST_M18 XRAYS_M18 MRI_M18  \
0            2.0      1.0

In [ ]:
for target_name, y in targets.items():
    print(f"\n Prediction: {target_name}")

    # Combine features and actual target column (y is a Series with the correct column)
    df_model = pd.concat([X, y], axis=1)
    #df_with_na = df_model[df_model.isna().any(axis=1)]
    #print(df_with_na)
    #num_na_rows = df_model.isna().any(axis=1).sum()
    #print(f"{num_na_rows} rows contain at least one NA value.")
    df_model = df_model.dropna()



    X_clean = df_model[feature_cols]
    y_clean = df_model[y.name]  # Get actual column name from Series

    X_train, X_test, y_train, y_test = train_test_split(
        X_clean, y_clean, test_size=0.2, random_state=42
    )

    for model_name, model in models.items():
        pipe = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])

        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        print(f"{model_name:20} | RMSE: {rmse:8.2f} | R²: {r2:.4f}")



 Prediction: Total Cost
